 The model is traned in Kaggle using this data set : <a href="https://www.kaggle.com/crawford/cat-dataset?rvi=1" >https://www.kaggle.com/crawford/cat-dataset?rvi=1</a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

Let's See a Sample Image  of a Cat with the points marked !

In [ ]:
img  =  cv2.cvtColor(cv2.imread('../input/cat-dataset/CAT_00/00000365_028.jpg'),cv2.COLOR_BGR2RGB)
img2 = cv2.resize(img,(300,300))

In [ ]:
file = open('../input/cat-dataset/CAT_00/00000365_028.jpg.cat')

In [ ]:
points = file.read().strip().split(' ')

In [ ]:
points = list(map(int,points))
x , y =  [] , []
for i in range(len(points[1:])) :
    if i%2 == 0 :
        x.append(points[1:][i])
    else :
        y.append(points[1:][i])

In [ ]:
x = np.asarray(x)
y = np.asarray(y)

In [ ]:
img.shape

In [ ]:
plt.imshow(img2)
plt.scatter(x/1024 *300,y/768 * 300,color='red')
plt.show()

In [ ]:
X = []
y = []

There are totally 7 folders containing the Cat photos with the landmark data from CAT_00 to CAT_06 ,
Since loading the whole dataset in numpy arrays wasn't possible , I trained them in batches , i.e I trained for 50 epochs with k in range 0 to 4 once 
and from k in range 4 to 6 the other time and repeated the process few times . 



In [ ]:
for k in range(4,7):
    
    j = 'CAT_0' + str(k)
    
  
    for i in os.listdir('../input/cat-dataset/'+j) :
        if '.jpg' not in i or '.cat' in i :
            continue
        img = cv2.imread('../input/cat-dataset/'+j+'/'+i)
        h , w , _ = img.shape
        img = cv2.resize(img,(300,300))
        
        X.append(img)
        
        file = open('../input/cat-dataset/'+j+'/'+i+'.cat')
    
        points = file.read().strip().split(' ')
        
        x_p , y_p =  [] , []
        
        points = list(map(int,points))
        
        if points[0] != 9 :
            print("Not 9")
            continue
            
        for l in range(len(points[1:])) :
            if l%2 == 0 :
                x_p.append(points[1:][l])
            else :
                y_p.append(points[1:][l])
                
        x_p = np.array(x_p)
        y_p = np.array(y_p)
        x_p = x_p / w * 300
        y_p = y_p /h * 300
        
                
        y.append(np.append(x_p,y_p))


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
model = tf.keras.Sequential([
    
    tf.keras.layers.Conv2D(64 ,(5,5), input_shape=(300,300,3),activation='relu'),
    tf.keras.layers.Conv2D(128,(5,5),activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(128,(5,5),activation='relu'),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),
    
    
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(1024,(3,3),activation='relu'),
    tf.keras.layers.Conv2D(1024,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.BatchNormalization(),
    
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(18),
    
    
])

In [ ]:
model.summary()

In [ ]:
model = tf.keras.models.load_model('model.h5')

In [ ]:
model.compile(optimizer='adam' , loss='mse' ,metrics=['mse','mae'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model.h5',save_best_only = True ,save_weights_only = False ,verbose=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train , X_test , y_train , y_test = train_test_split( X , y  ,test_size=0.1, random_state = 12)

In [ ]:
hist = model.fit(X_train,y_train , validation_data=(X_test ,y_test ),callbacks=[checkpoint] , epochs=50)